Importing libraries and get data from csv (imported as csv in part2)

In [1]:
import numpy as np
import pandas as pd
import json 
from pandas.io.json import json_normalize 

from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

import folium 


df=pd.read_csv('toronto_w_lat_lon.csv')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75188,-79.33036
1,M4A,North York,Victoria Village,43.73042,-79.31282
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302


Only Toronto!

In [2]:
toronto_df = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_df.shape)
toronto_df.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818
3,M5C,Downtown Toronto,St. James Town,43.65143,-79.37557
4,M4E,East Toronto,The Beaches,43.67703,-79.29542


In [3]:
address = 'Toronto, ONTARIO'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [4]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

for lat, lng, borough, Neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Defining Foursquare API

In [5]:
CLIENT_ID = 'yourID' # your Foursquare ID
CLIENT_SECRET = 'yourSecret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version, especially from 2018 :)

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NF1ZZVUYVSTRCUMVEDXOXELTM1RJ2SNOAY1B1N4N0L5UTSPY
CLIENT_SECRET:MCFP3CVFUXE0RRRJFGCEUJ1K2BIQ2XUS4JLSUAL210CXHMAP


Function for getting venues:

In [6]:
import requests

radius=500
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            

        results = requests.get(url).json()["response"]['groups'][0]['items']
        

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Here are Toronto venues:

In [7]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [8]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65514,-79.36265,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65514,-79.36265,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65514,-79.36265,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65514,-79.36265,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65514,-79.36265,Body Blitz Spa East,43.654735,-79.359874,Spa
5,"Regent Park, Harbourfront",43.65514,-79.36265,Rooster Coffee,43.651900,-79.365609,Coffee Shop
6,"Regent Park, Harbourfront",43.65514,-79.36265,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
7,"Regent Park, Harbourfront",43.65514,-79.36265,Sumach Espresso,43.658135,-79.359515,Coffee Shop
8,"Regent Park, Harbourfront",43.65514,-79.36265,Alumnae Theatre,43.652756,-79.364753,Theater
9,"Regent Park, Harbourfront",43.65514,-79.36265,Berkeley Church,43.655123,-79.365873,Event Space


In [9]:
toronto_venues.shape

(1741, 7)

In [10]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,68,68,68,68,68,68
"Brockton, Parkdale Village, Exhibition Place",86,86,86,86,86,86
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",68,68,68,68,68,68
Central Bay Street,71,71,71,71,71,71
Christie,11,11,11,11,11,11
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,26,26,26,26,26,26


In [11]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 226 uniques categories.


One hot encoding

In [12]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
toronto_onehot.shape

(1741, 226)

In [14]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.000000,0.000000,...,0.000000,0.00,0.000000,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.023256,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,...,0.000000,0.00,0.000000,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.010000,...,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014706,0.00
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.014085,0.000000,...,0.014085,0.00,0.000000,0.000000,0.014085,0.014085,0.014085,0.000000,0.000000,0.00
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Church and Wellesley,0.013333,0.000000,0.000000,0.013333,0.000000,0.000000,0.000000,0.000000,0.013333,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,"Commerce Court, Victoria Hotel",0.010000,0.000000,0.000000,0.040000,0.000000,0.000000,0.010000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.038462,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [15]:
toronto_grouped.shape

(39, 226)

In [16]:
num_top_venues = 5
for neigh in toronto_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1      Restaurant  0.04
2            Café  0.03
3  Breakfast Spot  0.03
4     Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.07
1          Bar  0.06
2  Coffee Shop  0.06
3   Restaurant  0.05
4    Gift Shop  0.03


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0         Coffee Shop  0.10
1               Hotel  0.05
2  Italian Restaurant  0.04
3                 Bar  0.03
4          Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.06
2                 Bar  0.04
3                Café  0.04
4   French Restaurant  0.03


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.11
1       

Most common venues

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(39, 11)

Clustering:

In [19]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 3, 0, 4, 0,
       0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [26]:
neighborhoods_venues_sorted.head()

,Cluster_Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Berczy Park,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Farmers Market,Seafood Restaurant,Beer Bar,Café,Cheese Shop,Bakery
1,0,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Sandwich Place,Gift Shop,Yoga Studio,Breakfast Spot,Burrito Place,Italian Restaurant
2,0,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Italian Restaurant,Restaurant,Bar,Asian Restaurant,Sushi Restaurant,Gastropub,Seafood Restaurant,Pub
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Bar,Café,Gym / Fitness Center,Restaurant,Park,Speakeasy,Intersection,French Restaurant
4,0,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Sushi Restaurant,Restaurant,Electronics Store,Bubble Tea Shop,Hotel,Middle Eastern Restaurant,Cosmetics Shop


In [25]:
toronto_grouped_clustering.head(10)

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0.014706,0.000000,0.0,0.000000,0.0,0.0,0.014706,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.014706,0.000000,0.000000,0.000000,0.0,0.000000,0.0
1,0.023256,0.011628,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.023256,0.011628,...,0.000000,0.0,0.0,0.011628,0.000000,0.000000,0.000000,0.0,0.000000,0.0
2,0.000000,0.000000,0.0,0.020000,0.0,0.0,0.010000,0.000000,0.010000,0.030000,...,0.000000,0.0,0.0,0.020000,0.000000,0.000000,0.010000,0.0,0.000000,0.0
3,0.014706,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.014706,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.014706,0.0
4,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.014085,0.014085,0.000000,0.000000,...,0.014085,0.0,0.0,0.000000,0.014085,0.014085,0.014085,0.0,0.000000,0.0
5,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
6,0.013333,0.000000,0.0,0.013333,0.0,0.0,0.000000,0.000000,0.013333,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
7,0.010000,0.000000,0.0,0.040000,0.0,0.0,0.010000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.010000,0.000000,0.000000,0.010000,0.0,0.000000,0.0
8,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.038462,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
9,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [24]:
#neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)
neighborhoods_venues_sorted.head(10)

,Cluster_Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Berczy Park,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Farmers Market,Seafood Restaurant,Beer Bar,Café,Cheese Shop,Bakery
1,0,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Sandwich Place,Gift Shop,Yoga Studio,Breakfast Spot,Burrito Place,Italian Restaurant
2,0,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Italian Restaurant,Restaurant,Bar,Asian Restaurant,Sushi Restaurant,Gastropub,Seafood Restaurant,Pub
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Bar,Café,Gym / Fitness Center,Restaurant,Park,Speakeasy,Intersection,French Restaurant
4,0,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Sushi Restaurant,Restaurant,Electronics Store,Bubble Tea Shop,Hotel,Middle Eastern Restaurant,Cosmetics Shop
5,0,Christie,Café,Grocery Store,Italian Restaurant,Athletics & Sports,Playground,Coffee Shop,Baby Store,Candy Store,Dumpling Restaurant,Eastern European Restaurant
6,0,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Bubble Tea Shop,Café,Hotel,Pub,Smoke Shop
7,0,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Beer Bar,Gym,American Restaurant,Japanese Restaurant,Seafood Restaurant,Italian Restaurant
8,0,Davisville,Dessert Shop,Pizza Place,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Gym,Tennis Court,Diner,Restaurant
9,0,Davisville North,Japanese Restaurant,Park,Food & Drink Shop,Department Store,Hotel,Breakfast Spot,Dance Studio,Cupcake Shop,Fast Food Restaurant,Farmers Market


In [21]:
toronto_merged_df = toronto_df

toronto_merged_df = toronto_df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged_df.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265,0,Coffee Shop,Breakfast Spot,Restaurant,Yoga Studio,Theater,Event Space,Spa,Electronics Store,Food Truck,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302,0,Coffee Shop,Sandwich Place,Park,Café,Museum,Salon / Barbershop,Chinese Restaurant,Restaurant,Clothing Store,Pub
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65736,-79.37818,0,Coffee Shop,Clothing Store,Café,Hotel,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Movie Theater,Bookstore,Diner
3,M5C,Downtown Toronto,St. James Town,43.65143,-79.37557,0,Coffee Shop,Seafood Restaurant,Café,Cocktail Bar,American Restaurant,Bakery,Moroccan Restaurant,Diner,Cosmetics Shop,Lingerie Store
4,M4E,East Toronto,The Beaches,43.67703,-79.29542,0,Health Food Store,Pub,Trail,Distribution Center,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Elementary School


Visualize:

In [22]:
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_df['Latitude'], toronto_merged_df['Longitude'], toronto_merged_df['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters

That's all!
